## Useful Links
- http://www.cpcb.gov.in/CAAQM/frmUserAvgReportCriteria.aspx
- http://www.cpcb.gov.in/CAAQM/mapPage/frmindiamap.aspx

### Make Use of Beautiful Soup to automate fetching data
- Historic Annual PM2.5, PM10 Plot
    - INDIA - National
    - Delhi NCR
    - Delhi Location wise
    - Gurgaon

###Mechanize Links:
   - http://www.search.sourceforge.net/mechanize/
   - Forms Example: http://www.search.sourceforge.net/mechanize/forms.html
   - BSoup Input Parsing: http://stackoverflow.com/questions/23001678/python-beautiful-soup-form-input-parsing

In [50]:
from bs4 import BeautifulSoup
import mechanize
import re
import time
import random

In [51]:
url = 'http://www.cpcb.gov.in/CAAQM/frmUserAvgReportCriteria.aspx'

In [52]:
random.seed()

In [53]:
def getOptions(param, form):
    return re.findall("{}=\[(.+)\]".format(param), str(form))

def getStations(ddlState, ddlCity):
    br = mechanize.Browser()
    br.set_handle_robots(False)
    res = br.open(url)
    forms = mechanize.ParseResponse(res, backwards_compat=False)
    form = forms[0]
    # State
    form["ddlState"] = ["{}".format(ddlState),]
    form, html = updateForm(br, form)
    # City
    form["ddlCity"] = ["{}".format(ddlCity),]
    form, html = updateForm(br, form)
    
    options = getOptions("ddlStation", form)
    options = options[0].split(',')[1:]
    options = map(lambda x: x.strip(), options)
    stations = options
    return stations

In [54]:
#     Example Usage: 
#     soup = BeautifulSoup(html)
#     getMap(soup, "ddlState")
def getMap(soup, ty):
    select_node = soup.findAll('select', attrs={'name': ty})
    if not select_node:
        select_node = soup.findAll('select', attrs={'id': ty})
#     print select_node
    option_map = {}
    if select_node:
        for option in select_node[0].findAll('option'):
            option_map[option['value']] = option.text
    return option_map

In [55]:
maxSleepTime = 2.0
def randSleepTime():
    return (1.0 + random.random() * maxSleepTime)

In [56]:
def getPM25Id(html):
    return re.findall("<option value=(.+)>(.*)PM2\.5(.*)</option>", str(html))

def getData(state, city):
    # Delhi, Delhi
#     state, city = "6", "85"
    stations = getStations(state, city)
    soups = []
    #
    random.shuffle(stations)
    for station in stations:
        try:
            br = mechanize.Browser()
            br.addheaders = [('user-agent', 
            '   Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.2.3) Gecko/20100423 Ubuntu/10.04 (lucid) Firefox/3.6.3'),
            ('accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8')]

            br.set_handle_robots(False)
            res = br.open(url)

            forms = mechanize.ParseResponse(res, backwards_compat=False)
            form = forms[0]

            form["ddlState"] = [state,]
            form, html = updateForm(br, form)

            form["ddlCity"] = [city,]
            form, html = updateForm(br, form)

            form["ddlStation"] = ["{}".format(station),]
            form, html = updateForm(br, form)

    #         print html
            parsed_pm25 = getPM25Id(html)
            if parsed_pm25:
                PM25Id = getPM25Id(html)[0][0][1:-1]
                print("PM2.5 Id =", PM25Id)
            else:
                print("No PM2.5 for {}, {}, {}").format(state, city, station)
                continue

            form["lstBoxChannelLeft"] = ["{}".format(PM25Id),]
            form, html = updateForm(br, form)

            form["ddlCriteria"] = ["0",]
            form, html = updateForm(br, form)

            form["txtYear"] = "2000"
            form["txtYearTo"] = "2017"
            form, html = updateForm(br, form)

            form["btnSubmit"] = "True"
            form, html = updateForm(br, form)

            br.select_form("form1")

            time.sleep(randSleepTime())
            res = br.submit(name='btnSubmit')
            html = res.read()
            soup = BeautifulSoup(html, 'html.parser')
            soups.append([(state, city, station), soup])
    #         if len(soups) >= 2:
    #             break
            time.sleep(randSleepTime())
            br.close()
            print ("Finished Crawling for {}, {}, {}").format(state, city, station)

        except:
            print("Exception raised for {}, {}, {}").format(state, city, station)
            time.sleep(5)
            br.close()
            continue
            
    return soups

In [45]:
def updateForm(br, form):
    time.sleep(randSleepTime())
    res = br.open(form.click())
    html = res.get_data()
    forms = mechanize.ParseResponse(res, backwards_compat=False)
    form = forms[0]
    return form, html

In [46]:
def getValsHtml(table):
    data = []
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    vals = re.findall("\d+\.\d+", data[0][0])[:-1]
    data = []
    for val in vals:
        yr = val[:4]
        data.append([yr, val[4:]])
    return data

def cleanupData(data):
    clean_data = []
    for elem in data:
        if elem[0] in map(str, range(2000, 2018)):
            clean_data.append(elem)
    return clean_data

In [47]:
import itertools
f = open('C:/Users/Scarlett RED/Desktop/cpcb_air_quality-master/id-mappings/city-ids.txt', 'r')
cities = f.readlines()#returns a list contains the lines of the file
print(cities)
#cities = list(map(lambda elem: elem.split(","))
         # print(cities[1:])
#cities = list(map(lambda elem: [elem[0], elem[1], elem[2], elem[3][:-1]])
 #             print(cities)
#cities = filter(lambda elem: elem[2].isdigit()) 
    

['stateId,state,cityId,city\n', '24,Sikkim,Select City,Select City\n', '25,Tamil Nadu,546,Chennai\n', '25,Tamil Nadu,Select City,Select City\n', '26,Tripura,Select City,Select City\n', '27,Uttaranchal,Select City,Select City\n', '20,Nagaland,Select City,Select City\n', '21,Orissa,Select City,Select City\n', '22,Punjab,194,Ludhiana\n', '22,Punjab,188,Amritsar\n', '22,Punjab,553,Mandi Gobindgarh\n', '22,Punjab,Select City,Select City\n', '23,Rajasthan,212,Jodhpur\n', '23,Rajasthan,223,Jaipur\n', '23,Rajasthan,Select City,Select City\n', '28,Uttar Pradesh,278,Kanpur\n', '28,Uttar Pradesh,253,Agra\n', '28,Uttar Pradesh,270,Varanasi\n', '28,Uttar Pradesh,256,Lucknow\n', '28,Uttar Pradesh,Select City,Select City\n', '29,West Bengal,300,Kolkata\n', '29,West Bengal,548,Howrah\n', '29,West Bengal,549,Haldia\n', '29,West Bengal,Select City,Select City\n', '29,West Bengal,552,Durgapur\n', '1,Andhra Pradesh,9,Visakhapatnam\n', '1,Andhra Pradesh,21,Tirupati\n', '1,Andhra Pradesh,Select City,Select 

In [48]:
cities=map(lambda elem:elem.split(","), cities[1:])
cities=map(lambda elem:[elem[0], elem[1], elem[2], elem[3][:-1]], cities)
cities=filter(lambda elem: elem[2].isdigit(), cities)

In [49]:
for elem in cities:
    stateId, stateName, cityId, cityName = elem
    f = open('data/{}_{}.txt'.format(stateName, cityName), 'w')
    try:
        soups = getData(stateId, cityId)
        for s in soups:
            state, city, station = s[0]
            f.write("state:{},city:{},station:{}\n".format(state, city, station))
            html_data = s[1].find(id="gvReportStation")
            if html_data:
                data = getValsHtml(html_data)
                data = cleanupData(data)
                for elem in data:
                    f.write(','.join(str(x) for x in elem) + "\n")
            else:
                f.write("None\n")

            f.write("\n")
        f.close()
        print("---")
        print("Finished Crawling city {}, {}").format(stateName, cityName)
        time.sleep(7) # Sleep for 7 seconds before continuing for next city
    except:
        print("Error occurred at {}, {}, {}, {}").format(stateId, stateName, cityId, cityName)
        break

Error occurred at {}, {}, {}, {}


AttributeError: 'NoneType' object has no attribute 'format'

In [32]:
states = getStatesMap()

In [105]:
soups = getData()

PM2.5 Id = 1341
PM2.5 Id = 508
PM2.5 Id = 1378
PM2.5 Id = 1290
PM2.5 Id = 1377
PM2.5 Id = 7
PM2.5 Id = 513
PM2.5 Id = 1376
PM2.5 Id = 507
PM2.5 Id = 514
No PM2.5 for 6, 85, 58
PM2.5 Id = 1316
Exception raised for 6, 85, 53
No PM2.5 for 6, 85, 51
PM2.5 Id = 509
PM2.5 Id = 506
No PM2.5 for 6, 85, 55


In [107]:
f = open('delhi-pm25-data.txt', 'w')
for s in soups:
    state, city, station = s[0]
    f.write("state:{},city:{},station:{}\n".format(state, city, station))
    html_data = s[1].find(id="gvReportStation")
    if html_data:
        data = getValsHtml(html_data)
        data = cleanupData(data)
        for elem in data:
            f.write(','.join(str(x) for x in elem) + "\n")
    else:
        f.write("None\n")
    
    f.write("\n")
f.close()

In [79]:
for soup in soups:
    table = soup[1].find(id="gvReportStation")
    if table:
        data = getValsHtml(table)
        data = cleanupData(data)
        print data
    else:
        print "No Data available for this station"

No Data available for this station
[[u'2016', u'216.32'], [u'2017', u'123.35']]


In [10]:
print getStations("6", "85")

['1', '2', '3', '4', '5', '6', '7', '45', '46', '47', '51', '52', '53', '54', '55', '58']
